# Práctica puertas cuánticas

<p align="center">
    <img src="https://i.pinimg.com/originals/b0/b7/64/b0b76439e5cd5ef9bab27e83c4fdb2f2.gif" alt="Quantum Gates Practice">
</p>

El  objetivo  de  la  presente  actividad  es  entender  el  funcionamiento  de  la  superposición  y  las  puertas  de 
control.

In [50]:
from pyquil import get_qc, Program
from pyquil.gates import I, H, X, MEASURE, CNOT
from pyquil.quilbase import Declare

In [2]:
def es_ganador(resultado, tiradas):
    jugadas_ganadas_j1 = 0
    jugadas_ganadas_j2 = 0
    empates = 0
    
    for i in resultado:
        # Jugador 1: cara(0), Jugador 2: cruz(1)
        if i[0] == 0 and i[1] == 1: 
            jugadas_ganadas_j1 += 1
        # Jugador 1: cruz(1), Jugador 2: cara(0)
        elif i[0] == 1 and i[1] == 0:  
            jugadas_ganadas_j2 += 1
        # Ambos sacan lo mismo
        else:  
            empates += 1
    
    print(f"Resultados después de {tiradas} tiradas:")
    print(f"  Jugador 1: {jugadas_ganadas_j1} victorias")
    print(f"  Jugador 2: {jugadas_ganadas_j2} victorias")
    print(f"  Empates: {empates}")
    print()
    
    if jugadas_ganadas_j1 > jugadas_ganadas_j2:
        print(f"Jugador 1 GANA con {jugadas_ganadas_j1} puntos")
    elif jugadas_ganadas_j2 > jugadas_ganadas_j1:
        print(f"Jugador 2 GANA con {jugadas_ganadas_j2} puntos")
    else:
        print(f"EMPATE: ambos con {jugadas_ganadas_j1} puntos")

In [22]:
def es_ganador_3_jugadores(resultado, tiradas):
    victorias_j1 = 0
    empates = 0
    
    for i in resultado:        
        if i[0] == 0 and i[1] == 1 and i[2] == 1:  
            victorias_j1 += 1
        elif i[0] == 1 and i[1] == 0 and i[2] == 0:  
            victorias_j1 += 1
        elif i[1] == 0 and i[0] == 0 and i[2] == 1:  
            empates += 1  
        elif i[1] == 1 and i[0] == 0 and i[2] == 1:  
            empates += 1
        else:
            empates += 1
    
    # Contar por jugador
    j1_caras = sum([1 for i in resultado if i[0] == 0])
    j1_cruces = sum([1 for i in resultado if i[0] == 1])
    j2_caras = sum([1 for i in resultado if i[1] == 0])
    j2_cruces = sum([1 for i in resultado if i[1] == 1])
    j3_caras = sum([1 for i in resultado if i[2] == 0])
    j3_cruces = sum([1 for i in resultado if i[2] == 1])
    
    print(f"  Jugador 1: {j1_caras} caras, {j1_cruces} cruces")
    print(f"  Jugador 2: {j2_caras} caras, {j2_cruces} cruces")
    print(f"  Jugador 3: {j3_caras} caras, {j3_cruces} cruces\n")
    
    # Verificación de trampas
    print("Verifar trampas")
    if j1_caras == tiradas:
        print("Jugador 1 SIEMPRE saca cara")
    if j3_cruces == tiradas:
        print("Jugador 3 SIEMPRE saca cruz")
    print(f"Jugador 2 es honesto: {j2_caras/tiradas*100:.0f}% caras, {j2_cruces/tiradas*100:.0f}% cruces")

## Parte 1

Simular el comportamiento aleatorio de ambos jugadores y comprobar quién vence en 50 tiradas. 

In [25]:
prog = Program(
    Declare('ro', 'BIT', 2),
    I(0), I(1),
    H(0), H(1),
    MEASURE(0, ('ro', 0)),
    MEASURE(1, ('ro', 1))
).wrap_in_numshots_loop(50)

qvm = get_qc('2q-qvm')
result = qvm.run(qvm.compile(prog)).get_register_map().get("ro")

es_ganador(
    resultado= result,
    tiradas=50
)

Resultados después de 50 tiradas:
  Jugador 1: 15 victorias
  Jugador 2: 14 victorias
  Empates: 21

Jugador 1 GANA con 15 puntos


## Parte 2

La segunda parte es donde el primer jugador será un tramposo. El segundo jugador debe seguir lanzando 
aleatoriamente la moneda, pero el primer jugador busca garantizarse ganar o empatar siempre, por lo que 
para su moneda debe salir siempre cara, partiendo de un estado inicial aleatorio. Buscamos construir el 
circuito y comprobar de nuevo quién vence en 50 tiradas (recordad, el segundo jugador no debería vencer 
nunca).


In [49]:
prog = Program(
    Declare('ro', 'BIT', 2),
    I(0),H(1),
    MEASURE(0, ('ro', 0)),  
    MEASURE(1, ('ro', 1))
).wrap_in_numshots_loop(50)

qvm = get_qc('2q-qvm')
result = qvm.run(qvm.compile(prog)).get_register_map().get("ro")

es_ganador(resultado=result, tiradas=50)

Resultados después de 50 tiradas:
  Jugador 1: 24 victorias
  Jugador 2: 0 victorias
  Empates: 26

Jugador 1 GANA con 24 puntos


## Parte 3

La última parte de la práctica incluye un tercer jugador. Este tercer jugador, que no está de acuerdo con las 
trampas realizadas en las partidas anteriores, busca el resultado contrario al segundo jugador, es decir, que 
siempre salga cruz. ¿Cómo puede garantizar este resultado partiendo del circuito resultante de la segunda 
parte?

In [82]:
prog = Program(
    Declare('ro', 'BIT', 3),
    
    I(0),H(1),              
    CNOT(1, 2),       
    X(2),              
    
    MEASURE(0, ('ro', 0)),
    MEASURE(1, ('ro', 1)),
    MEASURE(2, ('ro', 2)),
).wrap_in_numshots_loop(50)

qvm = get_qc('3q-qvm')
result = qvm.run(qvm.compile(prog)).get_register_map().get("ro")
es_ganador_3_jugadores(resultado=result, tiradas=50)

  Jugador 1: 50 caras, 0 cruces
  Jugador 2: 25 caras, 25 cruces
  Jugador 3: 25 caras, 25 cruces

Verifar trampas
Jugador 1 SIEMPRE saca cara
Jugador 2 es honesto: 50% caras, 50% cruces


## ¿Cómo puede garantizar este resultado partiendo del circuito resultante de la segunda parte?

Partiendo del circuito de la Parte 2, el Jugador 3 puede garantizar que siempre salga cruz utilizando las puertas CNOT(1, 2) seguida de X(2). La puerta CNOT copia el estado aleatorio del Jugador 2 (qubit 1) al qubit del Jugador 3 (qubit 2), creando un entrelazamiento entre ambos. Posteriormente, la puerta X invierte el estado copiado, de manera que si el Jugador 2 obtiene cara (0), el Jugador 3 obtendrá cruz (1), y viceversa. Sin embargo, si la intención es que el Jugador 3 obtenga siempre cruz independientemente del resultado del Jugador 2, basta con aplicar únicamente la puerta X(2) sobre el estado inicial |0⟩, lo cual lo transforma directamente en |1⟩, garantizando cruz en cada tirada sin necesidad de depender del resultado de otros jugadores.
